Tools in Langchain
1. how to create
2. how to use built in tools and toolkits
3. how to use chat models to call tools
4. how to pass tool outputs to chat models


This @tools decorator is the simplest way to define a custom tools. The decorator uses the function name as a tool name by default, but this can be overridden by passinf a string as a first argument. additionally, a decorator will be use the function's docstring as a tool's decription
- so docstring must be provided.

In [1]:
## @tool decorator

from langchain_core.tools import tool

@tool
def division(a:int,b:int)->int:
    """Divide 2 number"""
    return a/b

In [2]:
print(division.name)
print(division.description)
print(division.args)

division
Divide 2 number
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [3]:
## async implementation

from langchain_core.tools import tool


@tool
async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [4]:
from typing import Annotated,List

@tool
def multiply_by_max(
    a: Annotated[int,"A value"],
    b:Annotated[List[int],"list of ints over which to take maximum"]
)->int:
    """Mulitply a by the maximum of b"""
    return a* max(b)

In [5]:
print(multiply_by_max.args)

{'a': {'description': 'A value', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'list of ints over which to take maximum', 'items': {'type': 'integer'}, 'title': 'B', 'type': 'array'}}


### Structured Tool
The StructuredTool.from_function class method provides a bit more configurability than the @tool decorator, without requiring much additional code.

In [7]:
from langchain_core.tools import StructuredTool


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

calculator=StructuredTool.from_function(func=multiply,coroutine=amultiply)

print(calculator.invoke({"a": 2, "b": 3}))
print(await calculator.ainvoke({"a": 2, "b": 7}))

6
14


### Inbuilt tools

### wikipedia Integration

In [8]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper=WikipediaAPIWrapper(top_k_results=5,doc_content_chars_max=500)
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

print(tool.invoke({"query":"langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Vector database
Summary: A vector database, vector store or vector search engine is a database that stores and retrieves embeddings of data in v


In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [11]:
from langchain_tavily import TavilySearch

tool = TavilySearch(
    max_results=5,
    topic="general",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [12]:
tool.invoke("What is the recent ai news")

{'query': 'What is the recent ai news',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.artificialintelligence-news.com/',
   'title': 'AI News: Latest News, Analysis & Events',
   'content': 'Gartner Data & Analytics Summit unveils expanded AI agenda for 2026 · Thailand becomes one of the first in Asia to get the Sora app · Malaysia launches Ryt Bank',
   'score': 0.66777676,
   'raw_content': None},
  {'url': 'https://www.fastcompany.com/section/artificial-intelligence',
   'title': 'Artificial Intelligence',
   'content': "Latest Artificial Intelligence News. TECH · Google's threat intel chief explains why AI is now both the weapon and the target. As rivals and nation-states try",
   'score': 0.66464967,
   'raw_content': None},
  {'url': 'https://www.wsj.com/tech/ai?gaa_at=eafs&gaa_n=AWEtsqf5kOcqF4hgdF19uCvYV1IgBfjRNdqtxQfW8qpogveUAsP_Sw626NwV&gaa_ts=6997fd89&gaa_sig=YsxnceAAopNMih0t5VXcFN7ZRfp2blbrikWqyZDEMiCteWeNGDsMQhJb8cLMj62ZEVLx

In [13]:
from langchain_community.utilities import ArxivAPIWrapper

tool=ArxivAPIWrapper()

tool.run("1706.03762")

'Published: 2023-08-02\nTitle: Attention Is All You Need\nAuthors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin\nSummary: The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new 

### Call Tool With LLM Model

In [14]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print(wiki_tool.invoke({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


In [15]:
from langchain_tavily import TavilySearch

tavily_tool = TavilySearch(
    max_results=5,
    topic="general",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [16]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

In [17]:
wiki_tool.name

'wikipedia'

In [18]:

add.name

'add'

In [19]:

multiply.name

'multiply'

In [20]:
tavily_tool.name

'tavily_search'

In [21]:

tools=[wiki_tool,add,multiply,tavily_tool]

In [22]:

tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\langchain_tutorials\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=100)),
 StructuredTool(name='add', description='Adds a and b.', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x00000228E309B060>),
 StructuredTool(name='multiply', description='Multiplies a and b.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x00000228E309B100>),
 TavilySearch(max_results=5, topic='general', api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'), api_base_url=None))]

In [23]:
from langchain.chat_models import init_chat_model
llm=init_chat_model("qwen/qwen3-32b",model_provider="groq")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000228E33994D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000228E3504050>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [24]:
llm_with_tool=llm.bind_tools(tools)
llm_with_tool

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000228E33994D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000228E3504050>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'wikipedia', 'description': 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'query to look up on wikipedia', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function

In [25]:
from langchain_core.messages import HumanMessage
query="What is 2* 3"
messages = [HumanMessage(query)]
response=llm_with_tool.invoke(query)
print(response)

content='' additional_kwargs={'reasoning_content': 'Okay, let\'s see. The user is asking "What is 2* 3". First, I need to figure out what they\'re asking. The asterisk (*) is a multiplication symbol, so they want to know the product of 2 and 3.\n\nLooking at the tools provided, there\'s a multiply function. The description says it multiplies a and b, both integers. The parameters require a and b as integers. So I should use this function here.\n\nThe user input is straightforward. The numbers are 2 and 3. I need to call the multiply function with a=2 and b=3. There\'s no need for any other tools here because it\'s a simple arithmetic problem.\n\nI should check if there\'s any hidden complexity, like if the user meant something else, but the question seems direct. No need for Wikipedia or Tavily Search. The answer is 6, so the function call should return that.\n', 'tool_calls': [{'id': '0tgspv6rx', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}]} res

In [26]:
response

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, let\'s see. The user is asking "What is 2* 3". First, I need to figure out what they\'re asking. The asterisk (*) is a multiplication symbol, so they want to know the product of 2 and 3.\n\nLooking at the tools provided, there\'s a multiply function. The description says it multiplies a and b, both integers. The parameters require a and b as integers. So I should use this function here.\n\nThe user input is straightforward. The numbers are 2 and 3. I need to call the multiply function with a=2 and b=3. There\'s no need for any other tools here because it\'s a simple arithmetic problem.\n\nI should check if there\'s any hidden complexity, like if the user meant something else, but the question seems direct. No need for Wikipedia or Tavily Search. The answer is 6, so the function call should return that.\n', 'tool_calls': [{'id': '0tgspv6rx', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'funct

In [27]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': '0tgspv6rx',
  'type': 'tool_call'}]

In [28]:
for tool_call in response.tool_calls:
    selected_tool = {"add": add, "multiply": multiply,"wikipedia":wiki_tool,"tavily_search":tavily_tool}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
messages

[HumanMessage(content='What is 2* 3', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='6', name='multiply', tool_call_id='0tgspv6rx')]

In [29]:

messages

[HumanMessage(content='What is 2* 3', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='6', name='multiply', tool_call_id='0tgspv6rx')]

In [30]:
llm_with_tool.invoke(messages)

AIMessage(content='The result of multiplying 2 by 3 is **6**.', additional_kwargs={'reasoning_content': 'Okay, the user asked "What is 2* 3". Let me see. The tools available include a Wikipedia wrapper, an add function, a multiply function, and a Tavily search engine. The user is asking for the product of 2 and 3. Since there\'s a multiply function, I should use that. The parameters for multiply are a and b, both integers. So, a is 2 and b is 3. I need to call the multiply function with these values. The answer should be 6. Let me make sure there\'s no trick here, but 2 multiplied by 3 is straightforward. The other tools aren\'t needed here because it\'s a simple math problem. The response from the tool was 6, so I just need to present that as the answer.\n'}, response_metadata={'token_usage': {'completion_tokens': 184, 'prompt_tokens': 1947, 'total_tokens': 2131, 'completion_time': 0.367032235, 'completion_tokens_details': {'reasoning_tokens': 165}, 'prompt_time': 0.097847749, 'prompt

In [31]:
from langchain_core.messages import HumanMessage

query = "What is langchain and what is 5*15?"
messages = [HumanMessage(query)]

ai_msg = llm_with_tool.invoke(messages)
ai_msg

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking two separate questions here. First, "What is langchain?" and second, "what is 5*15?" Let me break this down.\n\nFor the first part, "langchain" probably refers to a company or a technology. Since it\'s a general knowledge question, I should use the Wikipedia function to get an accurate description. The user might be asking about the company LangChain, which I think is involved in AI or blockchain. I\'ll need to look that up.\n\nThen, the second part is a math problem: 5 multiplied by 15. That\'s straightforward arithmetic. The multiply function is available, so I can use that. The parameters are integers, so I just need to pass a=5 and b=15.\n\nWait, do I need to check if "langchain" is a common term or if there\'s a specific article? Maybe there\'s a Wikipedia page for LangChain. Also, I should make sure there\'s no confusion with similar names. Once I get the info from Wikipedia, I can summarize i

In [32]:
ai_msg.tool_calls

[{'name': 'wikipedia',
  'args': {'query': 'LangChain'},
  'id': '03dbgkv2m',
  'type': 'tool_call'},
 {'name': 'multiply',
  'args': {'a': 5, 'b': 15},
  'id': 'b49r77h2r',
  'type': 'tool_call'}]

In [33]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply,"wikipedia":wiki_tool,"tavily_search":tavily_tool}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
messages

[HumanMessage(content='What is langchain and what is 5*15?', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of ', name='wikipedia', tool_call_id='03dbgkv2m'),
 ToolMessage(content='75', name='multiply', tool_call_id='b49r77h2r')]

In [34]:

response=llm_with_tool.invoke(messages)

In [35]:
response.content

'LangChain is a software framework designed to facilitate the integration and orchestration of various tools, models, and data sources, enabling developers to build complex workflows and applications. It provides abstractions for tasks like retrieval-augmented generation, agent-based workflows, and multi-step pipelines. For more details, you can explore its Wikipedia page.\n\nAdditionally, $5 \\times 15 = 75$.'

In [36]:
from langchain_core.messages import HumanMessage

query = "What is langchain and what is 5*15 and summarize the recent AI news ?"
messages = [HumanMessage(query)]

ai_msg = llm_with_tool.invoke(messages)
ai_msg

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, let\'s break down the user\'s query. They asked three things: what is LangChain, calculate 5*15, and summarize recent AI news. \n\nFirst, for LangChain, I need to explain what it is. Since it\'s a tool or framework, maybe Wikipedia has a good overview. I should use the wikipedia function with a query like "LangChain".\n\nNext, the math problem 5*15. That\'s straightforward multiplication. The multiply function can handle this. The parameters would be a=5 and b=15.\n\nLastly, the recent AI news. This requires current information, so the tavily_search function is appropriate. The query would be "recent AI news". I should set the time_range to "week" to get the latest updates. Also, maybe set include_images to false since the user wants a summary, not images.\n\nWait, do I need to check if all functions are necessary? The user wants three separate pieces of info, so each should be handled by a different function. Let me 

In [37]:
ai_msg.tool_calls

[{'name': 'wikipedia',
  'args': {'query': 'LangChain'},
  'id': '7nvvptnw5',
  'type': 'tool_call'},
 {'name': 'multiply',
  'args': {'a': 5, 'b': 15},
  'id': 'a6at66jar',
  'type': 'tool_call'},
 {'name': 'tavily_search',
  'args': {'query': 'recent AI news',
   'search_depth': 'advanced',
   'time_range': 'week'},
  'id': 't96ekfb21',
  'type': 'tool_call'}]

In [38]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply,"wikipedia":wiki_tool,"tavily_search":tavily_tool}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
messages

[HumanMessage(content='What is langchain and what is 5*15 and summarize the recent AI news ?', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of ', name='wikipedia', tool_call_id='7nvvptnw5'),
 ToolMessage(content='75', name='multiply', tool_call_id='a6at66jar'),
 ToolMessage(content='{"query": "recent AI news", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.artificialintelligence-news.com/", "title": "AI News | Latest News | Insights Powering AI-Driven Business Growth", "content": "### PepsiCo is using AI to rethink how factories are designed and updated\\n\\nAI Business Strategy, Deep Dives, Featured News, Features, Governance, Regulation & Policy, Inside AI\\n\\n### Meeting the new ETSI standard for AI security\\n\\n## Latest\\n\\nAI Business Strategy\\n\\nFebruary 18, 2026\\n\\n# Infosys AI implementation framework offers

In [39]:
response=llm_with_tool.invoke(messages)
response.content


"1. **LangChain**: A software framework designed to integrate AI components, enabling developers to build applications using large language models (LLMs) and other AI tools. It provides a structured way to connect, manage, and orchestrate AI workflows.\n\n2. **5*15**: $ 5 \\times 15 = 75 $.\n\n3. **Recent AI News Summary**:\n   - **PepsiCo** is leveraging AI to redesign and optimize factory operations.\n   - **ETSI** (European Telecommunications Standards Institute) has introduced new standards for AI security.\n   - **Google** announced global partnerships and AI initiatives in India, including the Google Center for Climate Technology and fiber-optic connectivity projects.\n   - **Accenture** is linking employee promotions to the use of AI tools, reflecting growing industry adoption.\n   - **AI Market Trends**: Mixed performance in AI stocks (e.g., Microsoft, Oracle) due to unmet ROI expectations for generative AI projects, though long-term potential remains bullish.\n   - **Internati